## Lempel - Ziv complexity analysis 


Based on Schartner(2017) code. Calcuates LZs , LZc and Power spectral density based on the definitions from the 2017 paper. Uses MNE python for eeg data operations and minor changes to the original script prrovided by Schartner (2017). Input data uses Epoched EEGLAB .set files. 

Script adapted for the LZ correaltion experiment and to work with epoched data.



In [1]:
import mne
import LZ_Spectral_edited as fl
import numpy as np


from scipy import signal
from scipy.signal import (butter,lfilter,hilbert,resample)
from matplotlib.pylab import *
import os as os

# Channel selection for LZs and PSpec. OZ selected for best effect due to photic stim
channel = 31

# subject number
subject_number = ['1','2']
#subject_number = ['01','02','03','04','05','06','07','08','10','12','13','14','15','16','17','18','19','21','22','23']
subject_length = len(subject_number)
#Different Conditions 
condition = [11,12,13,21,22,23,31,32,33]
condition_length = len(condition)


LZS_data= np.zeros(len(condition))
LZC_data= np.zeros(len(condition))
#pspec_data = np.zeros([len(condition) + 1,5])
pspec_data = np.zeros(4)

array_len = (subject_length * condition_length)


data = np.zeros((array_len,5)) 


def LZc(X):
 '''
 Compute LZc and use shuffled result as normalization
 '''
 X=Pre2(X)
 SC=str_col(X)
 M=list(SC)
 shuffle(M)
 w=''
 for i in range(len(M)):
  w+=M[i]
 return cpr(SC)/float(cpr(w))

def Pre2(X):
 '''
 Linear-detrend and subtract mean of X, a multidimensional times series (channels x observations)
 '''
 ro,co=shape(X)
 Z=zeros((ro,co))
 for i in range(ro): #maybe divide by std?
  Z[i,:]=signal.detrend((X[i,:]-mean(X[i,:]))/std(X[i,:]), axis=0)
 return Z
 
  


def LZs(x):
 
 '''
 Lempel ziv complexity of single timeseries
 '''
 
 ro,co=np.shape(x)
 x=signal.detrend((x-mean(x))/std(x), axis=0)
 s=''
 r=abs(hilbert(x))
 th=mean(r)
 
 for j in range(co):
  if r[0,j]>th:
   s+='1'
  else:
   s+='0'
 
 M=list(s)
 shuffle(M)
 w=''
 for i in range(len(M)):
  w+=M[i]
 
 return cpr(s)/float(cpr(w))

  
def cpr(string):
 '''
 Lempel-Ziv-Welch compression of binary input string, e.g. string='0010101'. It outputs the size of the dictionary of binary words.
 '''
 d={}
 w = ''
 for c in string:
  wc = w + c
  if wc in d:
   w = wc
  else:
   d[wc]=wc   
   w = c
 return len(d)

def PSpec(X):
 '''
 X: multidimensional time series, ch x obs
 fs: sampling rate in Hz
 '''
 
 
 def find_closest(A, target):
   '''
   helper function
   '''
   #A must be sorted
   idx = A.searchsorted(target)
   idx = np.clip(idx, 1, len(A)-1)
   left = A[idx-1]
   right = A[idx]
   idx -= target - left < right - target
   return idx
 
 
 fs=1024
 
 de=[1,4]# in Hz
 th=[4,8]
 al=[8,13]
 be=[13,30]
# ga=[30,60]
# hga=[60,120]
 
 F=[de,th,al,be]#,ga,hga]
 
 ro,co=shape(X)
 Q=[]
 
 for i in range(ro):
 
  v=X[i]
  co=len(v)  
  N = co # Number of samplepoints  
  T = 1.0 / fs # sample spacing (denominator in Hz)
  y = v
  yf = fft(y)
  xf = np.linspace(0.0, 1.0/(2.0*T), int(N/2))
  yff=2.0/N * np.abs(yf[0:int(N/2)])
  bands=zeros(len(F))
  for i in range(len(F)):
   bands[i]=sum(yff[find_closest(xf, F[i][0]):find_closest(xf, F[i][1])])
  bands=bands/sum(bands)
  Q.append(bands)
 return Q


 
def str_col(X):
 '''
 Input: Continuous multidimensional time series
 Output: One string being the binarized input matrix concatenated comlumn-by-column
 '''
 ro,co=shape(X)
 TH=zeros(ro)
 M=zeros((ro,co))
 for i in range(ro):
  M[i,:]=abs(hilbert(X[i,:]))
  TH[i]=mean(M[i,:])
 
 s=''
 for j in range(co):
  for i in range(ro):
   if M[i,j]>TH[i]:
    s+='1'
   else:
    s+='0'
 
 return s
 
 

In [3]:
# Index variable for list and file saving
i = 0 

for y in subject_number:    
    for x in condition:        
        # Reset temp variables 
        LZC_temp = 0
        LZS_temp = 0
        # Loading indivdual epoched files
        filename = "0%s/pilot_%s_ICA_%s.set" % (y,y,x)
        data_file = mne.io.read_epochs_eeglab(filename)
        n_epochs = len(data_file)
        for index in range(n_epochs):
            fullset = np.reshape(data_file[index].get_data(),(64,4096))            
            specific_chandata = np.reshape(data_file[index].get_data(picks = 31),(1,4096))
            LZC_temp = LZC_temp + LZc(fullset)
            LZS_temp = LZS_temp + LZs(specific_chandata)
        
        # Assigning the data
        data [i,0] = y
        data [i,1] = x
        data [i,2] = LZC_temp / n_epochs
        data [i,3] = LZS_temp / n_epochs
        data [i,4] = n_epochs
        
        i = i+1

Extracting parameters from 01/pilot_1_ICA_11.set...
5 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from 01/pilot_1_ICA_12.set...
6 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from 01/pilot_1_ICA_13.set...
6 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from 01/pilot_1_ICA_21.set...
6 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from 01/pilot_1_ICA_22.set...
6 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting parameters from 01/pilot_1_ICA_23.set...
6 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.
Extracting param

In [4]:
# Saving the data to CSV file

np.savetxt('LZ_data.csv', data, delimiter =',',header = "participant,condition,LZC,LZS,N_epochs" )

In [ ]:
# Scratch Code below this point 

In [2]:
# Looking at the shape of the array 
print(array_len)
print(np.shape(data))

18
(18, 5)


In [ ]:
#Data test 
print (LZC_data)
print (LZS_data)

#ZZ = PSpec(data_file.get_data(picks = 31))
#LZC_data[0] = 'val'
#LZS_data[0] = 'val'

#print(ZZ)

In [ ]:
data_file = mne.io.read_raw_eeglab("21/021_43_ICA.set")

LZC_temp = LZc(data_file)
LZS_temp = LZs(data_file.get_data(picks = 31))


print(LZC_temp)
print(LZS_temp)

In [ ]:
print(fullset)

In [ ]:
fl.LZc(fullset)

In [ ]:
np.savetxt('test.txt',np.squeeze(fullset, axis=0), delimiter =',') 

In [23]:
np.shape(fullset)
#np.squeeze(specific_chandata, axis=0).shape

(1, 64, 4096)

In [ ]:
print(np.squeeze(fullset, axis=0))

In [35]:
filename = "02/pilot_2_ICA_11.set"
data_file = mne.io.read_epochs_eeglab(filename)
LZC_temp = 0
n_epochs = len(data_file)
test = np.zeros(n_epochs)
for index in range(n_epochs):
    fullset = np.reshape(data_file[index].get_data(),(64,4096))
    LZC_check = LZc(fullset)
    test[index] = LZC_check
    LZC_temp = LZC_temp + LZc(specific_chandata)
    
lzz = LZC_temp / n_epochs

Extracting parameters from 02/pilot_2_ICA_11.set...
44 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Ready.


In [36]:
print(test)

[0.09775518 0.09742043 0.09721098 0.09594758 0.09667551 0.09822917
 0.09631562 0.09646953 0.09914161 0.09716727 0.09539873 0.09617378
 0.09797662 0.09495723 0.09786181 0.09745896 0.09748852 0.09606731
 0.09822592 0.09873213 0.09589226 0.09834592 0.09571017 0.09621267
 0.09478652 0.09598164 0.09834299 0.0966785  0.09624107 0.09688877
 0.09737575 0.09746124 0.09695689 0.09804538 0.09974804 0.09605712
 0.09620322 0.09722222 0.0960734  0.09714851 0.09630761 0.09679603
 0.09817249 0.09563456]


In [39]:
print(lzz)

0.9568498208261843


In [34]:
test = 0

In [38]:
print(np.mean(test))

0.09697629274309509
